In [1]:
using DataFrames, XLSX
using LinearAlgebra
using JuMP
using LightGraphs, SimpleWeightedGraphs
using MathOptInterfaceMosek

┌ Info: Recompiling stale cache file C:\Users\cilli\.julia\compiled\v1.2\DataFrames\AR9oZ.ji for DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\cilli\.julia\compiled\v1.2\XLSX\gPxqz.ji for XLSX [fdbf4ff8-1666-58a4-91e7-1b58723a45e0]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\cilli\.julia\compiled\v1.2\LightGraphs\Xm08G.ji for LightGraphs [093fc24a-ae57-5d10-9952-331d41423f4d]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\cilli\.julia\compiled\v1.2\SimpleWeightedGraphs\iIsRI.ji for SimpleWeightedGraphs [47aef6b3-ad0c-573a-a1e2-d07658019622]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\cilli\.julia\compiled\v1.2\MathOptInterfaceMosek\sqIeN.ji for MathOptInterfaceMosek [0087ddc6-3964-5e57-817f-9937aefb0357]
└ @ Base loading.jl:1240


# Network Data

In [4]:
Branch = DataFrame(XLSX.readtable("3-Bus.xlsx", "BranchData", infer_eltypes=true)... )

,Branch,From,To,R,X,B,a,phi,Limit,Type
,Int64,Int64,Int64,Float64,Float64,Float64,Any,Any,Int64,String
1,1,1,2,0.0,0.1,10.0,missing,missing,150,line
2,2,1,3,0.0,0.125,8.0,missing,missing,1000,line
3,3,2,3,0.0,0.2,5.0,missing,missing,1000,line


### Define the bus set

In [5]:
Bus = max(maximum(Branch.From),maximum(Branch.To))

3

### Defining Susceptance Matrix (Laplacian)
##### Note: only Susceptance Matirx is defined for Linear power flow

In [6]:
Gw = SimpleWeightedGraph(Bus)

for i = 1:size(Branch,1)
    add_edge!(Gw,Branch.From[i],Branch.To[i], 1/Branch.X[i])
end

Aw = Array(weights(Gw))
Lw = Diagonal(sum(Aw,dims=1)[:])-Aw
B = Lw;

# Generator Location Data

In [7]:
#GenData = DataFrame(a = [0.001562, 0.00194, 0.00482], 
#                b = [7.92, 7.85, 7.97],
#                c = [561, 310, 78],
#                Pmax = [600, 400, 200],
#                Pmin = [150, 100, 50],
#                RampUp = [30,20,30],
#                RampDown = [30,20,30],
#                MinUp = [2,3,2],
#                MinDown = [2,3,2]);

In [8]:
GenData = DataFrame(XLSX.readtable("3-Bus.xlsx", "GenData", infer_eltypes=true)... )

,Name,Type,a,b,c,Pmax,Pmin,RampUp,RampDown,MinUp
,Int64,String,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,Unknown,0.001562,7.92,561,600,150,30,30,2
2,2,Unknown,0.00194,7.85,310,400,100,20,20,3
3,3,Unknown,0.00482,7.97,78,200,50,30,30,2


In [39]:
StorageData = DataFrame(XLSX.readtable("3-Bus.xlsx", "StorageData", infer_eltypes=true)... )

,Name,Type,ne,ni,C,a
,Int64,String,Float64,Float64,Int64,Int64
1,1,Unknown,0.9,0.9,600,0
2,2,Unknown,0.85,0.85,400,0
3,3,Unknown,0.8,0.8,200,0


In [9]:
PVar = DataFrame(Bus = [1, 2, 3], Gen = [1,1,1])

Bus = max(maximum(Branch.From),maximum(Branch.To))
Genw = SimpleDiGraph(Bus)

for i = 1:size(PVar,1)
    add_edge!(Genw,PVar.Bus[i],PVar.Gen[i] )
    println(PVar.Bus[i],PVar.Gen[i])
end
AGenw = Array(adjacency_matrix(Genw))

11
21
31


3×3 Array{Int64,2}:
 1  0  0
 1  0  0
 1  0  0

### Define Generator Set

In [10]:
Gen = size(GenData, 1)

3

# Load Data

In [11]:
Load = DataFrame( XLSX.readtable("3-Bus.xlsx", "LoadData", infer_eltypes=true)... )

,L1,L2,L3
,Int64,Int64,Int64
1,180,520,120
2,200,500,80
3,220,480,100


### Defing the max time period ( T )

In [12]:
T = size(Load,1)

3

In [13]:
LineLimit = [150, 1000, 1000];

# Optimization Model

In [49]:
model = nothing

In [50]:
model = Model(with_optimizer(MosekOptimizer, QUIET=false, INTPNT_CO_TOL_DFEAS=1e-7));

In [51]:
Bus = size(B,2); Gen =size(GenData,1) ;T = 3
@variable(model, P[1:Bus,1:Gen,1:T]);

In [52]:
for i in 1:Bus, j in 1:Gen

 if AGenw[i,j] == 0
    
      for t =1:T  
        fix( P[ i, j, t], 0, force = true);
      end
                
 else
 println(string("Gen",j ," at Bus", i))

 end

end

Gen1 at Bus1
Gen1 at Bus2
Gen1 at Bus3


In [53]:
@variable(model, PBus[1:Bus,1:T])
@variable(model, u[1:Bus,1:Gen,1:T],Bin)
@variable(model, Us[1:Bus,1:Gen,1:T],Bin)
@variable(model, Vs[1:Bus,1:Gen,1:T],Bin)
@variable(model, θ[1:Bus,1:T])
for t = 1:T 
    fix( θ[1,t], 0, force = true);
end

In [17]:
@objective(model,Min, sum(GenData.a[j]*P[ i, j, t]^2 + GenData.b[j]*P[ i, j, t] + GenData.c[j] 
         for t in 1:T for i in 1:Bus for j in 1:Gen ));

In [18]:
@constraint(model,LoadBalance[t = 1:T], sum(Load[t,i]   for i in 1:3) == sum(PBus[i,t] for i in 1:Bus));

In [19]:
@constraint(model,PowerFlow[i = 1:Bus, t = 1:T], 
    sum(100 * B[i,j] * (θ[i,t] - θ[j,t]) for j in 1:Bus)  == PBus[i,t]  - Load[t,i] );

In [20]:
@constraint(model,BusPower[i = 1:Bus, t = 1:T],  sum(P[i, g, t] for g in 1:Gen) == PBus[i,t]);

In [22]:
@constraint(model,MaxPowerLimits[i = 1:Bus,j=1:Gen,t = 1:T],  P[i,j,t] <= GenData.Pmax[j]*u[i,j,t])
@constraint(model,MinPowerLimits[i = 1:Bus,j=1:Gen,t = 1:T],  P[i,j,t] >= GenData.Pmin[j]*u[i,j,t]);

In [24]:
@constraint(model,MaxLineLimit[i = 1:Bus, t = 1:T],sum(100 * B[i,j] * (θ[i,t] - θ[j,t]) for j in 1:Bus) <=  Branch.Limit[i])
@constraint(model,MinLineLimit[i = 1:Bus, t = 1:T],sum(100 * B[i,j] * (θ[i,t] - θ[j,t]) for j in 1:Bus) >= -Branch.Limit[i]);

3×3 Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,2}:
 MinLineLimit[1,1] : -1800 θ[1,1] + 1000 θ[2,1] + 800 θ[3,1] >= -150.0   …  MinLineLimit[1,3] : -1800 θ[1,3] + 1000 θ[2,3] + 800 θ[3,3] >= -150.0 
 MinLineLimit[2,1] : -1500 θ[2,1] + 1000 θ[1,1] + 500 θ[3,1] >= -1000.0     MinLineLimit[2,3] : -1500 θ[2,3] + 1000 θ[1,3] + 500 θ[3,3] >= -1000.0
 MinLineLimit[3,1] : -1300 θ[3,1] + 800 θ[1,1] + 500 θ[2,1] >= -1000.0      MinLineLimit[3,3] : -1300 θ[3,3] + 800 θ[1,3] + 500 θ[2,3] >= -1000.0 

In [25]:
@constraint(model,RampUpLimit[i = 1:Bus, j = 1:Gen, t = 2:T], P[i,j,t] - P[i,j,t-1] <=  GenData.RampUp[i]*u[i,j,t])
@constraint(model,RampDownLimit[i = 1:Bus, j = 1:Gen, t = 2:T], P[i,j,t-1] - P[i,j,t] <=  GenData.RampDown[i]*u[i,j,t]);

In [26]:
@constraint(model,UpStatus[i = 1:Bus, j = 1:Gen, t = 2:T], u[i,j,t-1] - u[i,j,t] + Us[i,j,t] >=  0 );
@constraint(model,DownStatus[i = 1:Bus, j = 1:Gen, t = 2:T], u[i,j,t] - u[i,j,t-1] + Vs[i,j,t] >=  0 );

In [27]:
@constraint(model,MinUp[i = 1:Bus, j=1:Gen, t = 2:T, k = t+1:min(t+GenData.MinUp[i]-1, T) ], 
    u[i,j,t] - u[i,j,t-1]  <=  u[i,k,t] );
@constraint(model,MinDown[i = 1:Bus, j = 1:Gen, t = 2:T, k = t+1:min(t+GenData.MinDown[i]-1, T) ],
    u[i,j,t-1] - u[i,j,t]  <=  1 - u[i,k,t] );

### Linear Energy Storage (Feasible Set)

In [54]:
Storage = size([1,1],1) # Replace [1,1] with actual table

2

In [55]:
@variable(model, s[1:Bus,1:Storage,1:T]);
@variable(model, ue[1:Bus,1:Storage,1:T]);
@variable(model, ui[1:Bus,1:Storage,1:T]);

In [57]:
@constraint(model, SOC[i = 1:Bus, j = 1:Storage, t = 2:T], 
    s[i,j,t] == StorageData.a[j]*s[i,j,t-1] + (StorageData.ne[j]*ue[i,j,t-1] - ui[i,j,t-1]/StorageData.ni[j]));

In [58]:
@constraint(model, Capacity[i = 1:Bus, j = 1:Storage, t = 2:T],
    0 <=  s[i,j,t] <=  StorageData.C[j]);

In [59]:
@constraint(model, PowerStorage[i = 1:Bus, j = 1:Storage, t = 2:T],
    P[i,j,t] == ui[i,j,t] - ue[i,j,t]);

In [63]:
@constraint(model, GenEnergyMin[i = 1:Bus, j = 1:Storage, t = 2:T], 
    0 <= ue[i,j,t] );

In [64]:
@constraint(model, GenEnergyMax[i = 1:Bus, j = 1:Storage, t = 2:T], 
    ue[i,j,t] <= - P[i,j,t] );

In [65]:
@constraint(model, StoredEnergyMin[i = 1:Bus, j = 1:Storage, t = 2:T], 
    0 <= ui[i,j,t] );

In [66]:
@constraint(model, StoredEnergyMax[i = 1:Bus, j = 1:Storage, t = 2:T], 
   ui[i,j,t] <= P[i,j,t] );

In [28]:
model

A JuMP Model
Minimization problem with:
Variables: 126
Objective function type: GenericQuadExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 21 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 72 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 90 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 21 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 81 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek
Names registered in the model: BusPower, DownStatus, LoadBalance, MaxLineLimit, MaxPowerLimits, MinDown, MinLineLimit, MinPowerLimits, MinUp, P, PBus, PowerFlow, RampDownLimit, RampUpLimit, UpStatus, Us, Vs, u, θ

In [29]:
optimize!(model)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 212             
  Cones                  : 1               
  Scalar variables       : 156             
  Matrix variables       : 0               
  Integer variables      : 81              

Optimizer started.
Mixed integer optimizer started.
Threads used: 4
Presolve started.
Presolve terminated. Time = 0.00
Presolved problem: 44 variables, 54 constraints, 132 non-zeros
Presolved problem: 0 general integer, 9 binary, 35 continuous
Clique table size: 18
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        0        0        NA                   2.8549096533e+04     NA          0.1   
0        1        0        0        2.8549096533e+04     2.8549096533e+04     1.27e-14    0.1   
An optimal solution satisfying the relative gap tolerance of 1.0

In [ ]:
size(P)

In [ ]:
100*B[2,1]*(value(θ[2])-value(θ[1]))

In [ ]:
100*B[1,2]*(value(θ[1])-value(θ[2]))

In [ ]:
value.(PBus)

In [ ]:
Load

In [ ]:
sum(Load[1,:]) == sum(value.(PBus[:,1]))

In [ ]:
sum(Load[1,:]) == sum(value.(PBus[:,1]))

In [ ]:
value.(PBus[1,1])

In [ ]:
value.(P[1,:,1])